DRKG

Adapted from: https://github.com/gnn4dr/DRKG/blob/master/drkg_with_dgl/loading_drkg_in_dgl.ipynb

In [4]:
import pandas as pd
import numpy as np
import os 
import torch_geometric.transforms as T
import torch
from sklearn.metrics import accuracy_score

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric as pyg
import torch_geometric
from torch_geometric.nn import SAGEConv, to_hetero
from   torch.utils.data      import Dataset, DataLoader
from   torch_geometric.data  import Data
from   torch_geometric.utils import negative_sampling


from torch_geometric.nn           import SAGEConv, to_hetero
from SIMP_LLM.dataloader_mappings import load_graph,load_csv_as_list


from SIMP_LLM.models              import  GATStack,GNNStack,LinkPredictorMLP,CosineSimilarityModel,save_torch_model
from SIMP_LLM.train_utils         import  forward_pass, train,evaluate,   filter_edge_index_by_value,split_edge_index,get_negative_edges

import matplotlib.pyplot as plt #needed to visualize loss curves
import numpy as np 

from sklearn         import metrics
from sklearn.metrics import ConfusionMatrixDisplay
from  utils.pinecone_driver import PineCone_Driver
from config import  PINE_CONE_KEY, PINE_CONE_ENV,INDEX_NAME 


import copy

/opt/conda/envs/simp/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load Graph

In [5]:
#triplets =[('Compound', 'Compound_treats_the_disease', 'Disease')] # Uncomment this line 
triplets     = load_csv_as_list('triplets.csv')                     # triplets of form (tail, relationship,head)
data         = load_graph(triplets )                                # Load Graph based on triplets (You can selet the triplets 
dictionaries = torch.load("data2/mapping_dict")
#print(data)
#print(data["Compound", "Compound_treats_the_disease", "Disease"].edge_index)

data2


## GRAPH SAGE

#### 1 Masking Sanity Check

In [6]:
edge_index = torch.tensor([[0, 0, 0, 1, 1, 2, 2, 3,4,5,5,5,5,5], [1, 2, 3, 4, 5, 6, 7, 8,9,10,11,12,13,14]])
print(f"Oringal Edge index:\n{edge_index }\n: {edge_index.shape}\n")
edge_index_1 , edge_index_2 = split_edge_index(edge_index ,percentage = 0.5,verbose=True)

Oringal Edge index:
tensor([[ 0,  0,  0,  1,  1,  2,  2,  3,  4,  5,  5,  5,  5,  5],
        [ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14]])
: torch.Size([2, 14])

##############
Edge Index 1: torch.Size([2, 5])

Edge Index 2: torch.Size([2, 9])
##############


### 2 Mask Real Data 

In [5]:
## SPLIT TRAIN, TEST, VAL
edge_index_train , edge_index_validation_test          = split_edge_index(edge_index = data["Compound", "Compound_treats_the_disease", "Disease"].edge_index ,percentage = 0.9,verbose=True)
edge_index_validation ,edge_index_test                 = split_edge_index(edge_index = edge_index_validation_test  ,percentage = 0.5,verbose=True)


## Furthere split into EMBED and PREDICT for every split ###
edge_index_train_embed , edge_index_train_predict      = split_edge_index(edge_index = edge_index_train ,percentage = 0.8,verbose=True)
edge_index_val_embed   , edge_index_val_predict        = split_edge_index(edge_index = edge_index_validation   ,percentage = 0.85,verbose=True)
edge_index_test_embed  , edge_index_test_predict       = split_edge_index(edge_index = edge_index_validation   ,percentage = 0.85,verbose=True)





### Train ###
data_train_embed = copy.deepcopy(data)
data_train_embed["Compound", "Compound_treats_the_disease", "Disease"].edge_index = edge_index_train_embed
print(data_train_embed["Compound", "Compound_treats_the_disease", "Disease"].edge_index.shape)

data_train_predict = copy.deepcopy(data)
data_train_predict["Compound", "Compound_treats_the_disease", "Disease"].edge_index = edge_index_train_predict 
print(data_train_predict["Compound", "Compound_treats_the_disease", "Disease"].edge_index.shape)

### Val ###
data_val_embed = copy.deepcopy(data)
data_val_embed["Compound", "Compound_treats_the_disease", "Disease"].edge_index = edge_index_val_embed 
print(data_val_embed["Compound", "Compound_treats_the_disease", "Disease"].edge_index.shape)

data_val_predict = copy.deepcopy(data)
data_val_predict["Compound", "Compound_treats_the_disease", "Disease"].edge_index = edge_index_val_predict
print(data_val_predict["Compound", "Compound_treats_the_disease", "Disease"].edge_index.shape)



### Test ###
data_test_embed = copy.deepcopy(data)
data_test_embed["Compound", "Compound_treats_the_disease", "Disease"].edge_index = edge_index_test_embed 
print(data_test_embed["Compound", "Compound_treats_the_disease", "Disease"].edge_index.shape)

data_test_predict = copy.deepcopy(data)
data_test_predict["Compound", "Compound_treats_the_disease", "Disease"].edge_index = edge_index_test_predict
print(data_test_predict["Compound", "Compound_treats_the_disease", "Disease"].edge_index.shape)




##############
Edge Index 1: torch.Size([2, 40666])

Edge Index 2: torch.Size([2, 7888])
##############
##############
Edge Index 1: torch.Size([2, 1969])

Edge Index 2: torch.Size([2, 5919])
##############
##############
Edge Index 1: torch.Size([2, 30998])

Edge Index 2: torch.Size([2, 9668])
##############
##############
Edge Index 1: torch.Size([2, 972])

Edge Index 2: torch.Size([2, 997])
##############
##############
Edge Index 1: torch.Size([2, 972])

Edge Index 2: torch.Size([2, 997])
##############
torch.Size([2, 30998])
torch.Size([2, 9668])
torch.Size([2, 972])
torch.Size([2, 997])
torch.Size([2, 972])
torch.Size([2, 997])


###  3 Compile Model 

In [11]:
epochs        = 500
hidden_dim    = 524      
dropout       = 0.7
num_layers    = 3
learning_rate = 1e-4
node_emb_dim  = 768
device        = "cpu"
head          = "COSINE"    # one of   "MLP", "COSINE"
backbone      = "GraphSage" # one of  "GraphSage","GAT"



#GATStack,GNNStack


if backbone == "GraphSage":
    HomoGNN         = GNNStack(node_emb_dim, hidden_dim, hidden_dim, num_layers, dropout, return_embedding=True).to(device) # the graph neural network that takes all the node embeddings as inputs to message pass and agregate
elif backbone == "GAT":
    HomoGNN         = GATStack(node_emb_dim, hidden_dim, hidden_dim, num_layers, dropout, return_embedding=True).to(device) 
    
embed_model     = to_hetero(HomoGNN   , data.metadata(), aggr='sum')
if head == "MLP":
    link_predictor  = LinkPredictorMLP(hidden_dim, hidden_dim, 1, num_layers , dropout).to(device) # the MLP that takes embeddings of a pair of nodes and predicts the existence of an edge between them
if head == "COSINE":
    link_predictor = CosineSimilarityModel(input_dim=524 ).to(device)

#optimizer      = torch.optim.AdamW(list(model.parameters()) + list(link_predictor.parameters() ), lr=learning_rate, weight_decay=1e-4)
if head ==  "MLP":
    optimizer       = torch.optim.Adam(list(embed_model.parameters()) + list(link_predictor.parameters() ), lr=learning_rate)
elif head == "COSINE":
    optimizer       = torch.optim.Adam(list(embed_model.parameters()) , lr=learning_rate)
    

#print(HeteroGNN )
print(link_predictor)
print(f"Models Loaded to {device}")
data.to(device)
embed_model.to(device)
print()

/opt/conda/envs/simp/lib/python3.11/site-packages/torch_geometric/nn/to_hetero_transformer.py:165: UserWarning: The type 'Side Effect' contains invalid characters which may lead to unexpected behavior. To avoid any issues, ensure that your types only contain letters, numbers and underscores.
  warnings.warn(
/opt/conda/envs/simp/lib/python3.11/site-packages/torch_geometric/nn/to_hetero_transformer.py:165: UserWarning: The type 'Biological Process' contains invalid characters which may lead to unexpected behavior. To avoid any issues, ensure that your types only contain letters, numbers and underscores.
  warnings.warn(
/opt/conda/envs/simp/lib/python3.11/site-packages/torch_geometric/nn/to_hetero_transformer.py:165: UserWarning: The type 'Cellular Component' contains invalid characters which may lead to unexpected behavior. To avoid any issues, ensure that your types only contain letters, numbers and underscores.
  warnings.warn(
/opt/conda/envs/simp/lib/python3.11/site-packages/torch_

CosineSimilarityModel(
  (cos): CosineSimilarity()
)
Models Loaded to cpu



In [12]:
#### Load Best Models ####
best_graphsage_model_path      = os.path.join("Saved_models","GraphSageCosine","GraphSage_epoch_58.pt")
best_link_predictor_model_path = os.path.join("Saved_models","GraphSageCosine","link_predictor_epoch_58.pt")
print(f"Loading best models:  {best_graphsage_model_path }  {best_link_predictor_model_path}")
                                              
                                              
checkpoint = torch.load(best_graphsage_model_path  )
embed_model.load_state_dict(checkpoint['model_state_dict'])

checkpoint = torch.load(best_link_predictor_model_path)
link_predictor.load_state_dict(checkpoint['model_state_dict'])

del checkpoint


Loading best models:  Saved_models/GraphSageCosine/GraphSage_epoch_58.pt  Saved_models/GraphSageCosine/link_predictor_epoch_58.pt


In [13]:

import pinecone

class PineCone_Driver:
    def __init__(self,index_name , key, env,embed_dim:int=None,metric='cosine'):
        
        self.embed_dim  = embed_dim
        self.metric     = metric
        self.index_name = index_name
        self.key        = key
        self.env        = env
        self._init_pincone()
        self._get_index()


    def _init_pincone(self):
        pinecone.init(api_key=self.key, environment=self.env)

    def _get_index(self):
        if self.index_name not in pinecone.list_indexes():
            pinecone.create_index(name      =  self.index_name,
                                  dimension = self.embed_dim,
                                  metric    = self.metric)
            
        self.index = pinecone.Index(self.index_name) #pinecone.Index(self.index_name )


    def semantic_query(self, query, top_k:int=10, include_metadata=True,namespace:str="") -> list:
        return self.index.query(query, top_k=top_k, include_metadata=include_metadata,namespace=namespace)
    
    def print_semantic_query_result(self, result:list):
        for result_ in result['matches']:
            print(f"{round(result_['score'], 2)}: {result_['metadata']['text']}")
            
    def get_most_similar(self, query, top_k:int=10, include_metadata=True,threshold:float=0.80,namespace:str="") -> list:
        result = self.semantic_query(query, top_k=top_k, include_metadata=include_metadata,namespace=namespace)
        return [r for r in result['matches'] if r['score'] > threshold]


    def push(self, ids:list,embeddings:list,metadatas:list,namespace="default"):
        records = zip(ids, embeddings, metadatas)                          # create records list for upsert
        self.index.upsert(vectors=records,namespace=namespace)
   
    def fetch(self, ids:list):
        return self.index.fetch(ids)
    

    def decribe_database(self):
       print(self.index.describe_index_stats())




In [14]:
INDEX_NAME  = "bmi-212-pd-db"  

In [15]:
PineConeIndex = PineCone_Driver( index_name = INDEX_NAME   ,embed_dim= hidden_dim,  key = PINE_CONE_KEY, env = PINE_CONE_ENV,metric='cosine')

In [16]:
#embedded_nodes = embed_model(data_test_embed .x_dict, data_test_embed .edge_index_dict)
embedded_nodes = embed_model(data.x_dict, data.edge_index_dict)

In [17]:

dictionaries = torch.load("data2/mapping_dict")

In [18]:


def invert_dictionary(dictionary):
    inverted_dict = {value: key for key, value in dictionary.items()}
    return inverted_dict

def prepare_embeddings(embedded_nodes,name,dic_):
    embeddings = []
    ids        = []
    names      = []
    dic_       = invert_dictionary(dic_[name])
    embeded_   = F.normalize(embedded_nodes[name], p=2, dim=1)
    
    for id_, node in enumerate(embeded_):
        node  =   node.tolist()
        name  =   {"name":dic_[id_],"id":id_}
        embeddings.append(node)
        names.append(name)
        ids.append(f"{id_}")
    return embeddings,ids,names



   




In [12]:
import tqdm

In [13]:
EMBED_ENTITES = ['Compound','Disease', 'Gene']
batch_size    = 50
for ENTITY in EMBED_ENTITES: 
    embeded_compounds,ids,names = prepare_embeddings(embedded_nodes = embedded_nodes,name = ENTITY,dic_=dictionaries)
    
    for start in tqdm.tqdm(range(0, len(ids), batch_size)):
        end         = start + batch_size
        id_batch    = ids[start:end]
        embed_batch = embeded_compounds[start:end]
        name_batch = names[start:end]
        PineConeIndex.push(ids = id_batch, embeddings = embed_batch, metadatas = name_batch, namespace= ENTITY )

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 414/414 [00:45<00:00,  9.10it/s]


In [19]:
tensor = data.edge_index_dict[('Compound','Compound_treats_the_disease','Disease')]
tensor

tensor([[12303,  1300,  1300,  ..., 10757,  1532,  1577],
        [ 2567,  1700,  2452,  ...,  1127,  2891,  4076]])

In [20]:
filtered_tensors = tensor[:, tensor[0, :] == 95]

print(filtered_tensors)

tensor([[  95,   95,   95,   95,   95,   95,   95,   95,   95,   95,   95,   95,
           95,   95,   95,   95,   95,   95,   95,   95,   95,   95,   95,   95,
           95,   95,   95,   95,   95,   95,   95,   95,   95,   95,   95,   95,
           95,   95,   95,   95,   95,   95,   95,   95,   95,   95,   95,   95,
           95,   95,   95,   95,   95,   95,   95,   95,   95,   95,   95,   95,
           95,   95,   95,   95,   95,   95,   95,   95,   95,   95,   95,   95],
        [2335, 2510, 2366, 1543, 1537, 1725, 1724, 2434, 3042, 2309, 2769, 3729,
         4065, 1052, 2335, 1910, 2638, 1387, 1439, 2242,  367,  223,  684, 1483,
         1720, 2499, 2099, 2699,  942, 2426, 3270, 1192,  207,  625, 2100, 1073,
         1382, 2023, 2205, 3788, 1092,  140, 3233, 3592, 1942, 2143, 1420, 1543,
         2366, 2511, 2017, 1079, 1212, 1958, 2974,  695, 3116, 1447,  941,  809,
         1964, 1907, 2767, 3621, 1570, 3240, 1946, 1877, 3536, 2462, 2078, 1349]])


## Do look up

In [21]:
 embeded_compounds,ids,names = prepare_embeddings(embedded_nodes = embedded_nodes,name = 'Compound',dic_=dictionaries)

In [22]:
PineConeIndex.get_most_similar( query=embeded_compounds[95], top_k=3, include_metadata=True,threshold=0.70,namespace='Compound')

[{'id': '12120',
  'metadata': {'id': 12120.0, 'name': 'Esomeprazole'},
  'score': 0.878518641,
  'values': []},
 {'id': '12195',
  'metadata': {'id': 12195.0, 'name': 'Iopamidol'},
  'score': 0.87654233,
  'values': []},
 {'id': '5770',
  'metadata': {'id': 5770.0, 'name': 'Ketotifen'},
  'score': 0.876348317,
  'values': []}]

In [116]:
filtered_tensors = tensor[:, tensor[0, :] == 11171]
print(filtered_tensors)

tensor([[11171, 11171],
        [ 1147,  3798]])
